In [ ]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
excel_file_path = '/content/sample_data/8-9 to 10-11 Sep.xlsx'
xls = pd.ExcelFile(excel_file_path)
available_sheets = xls.sheet_names
channelSheets = {
    "Azaad Digital": ["Azaad Digital"],
    "Pukhtun Digital": ["Pukhtun Digital"],
    "Kashmir Digital": ["Kashmir Digital"],
    "Burak": ["Burak New", "Burak", "Burak "]
}
formattedChannelData = pd.DataFrame(columns=["Page", "Total Posts", "Impressions", "Followers", "Channel", "Platform", "Date", "Post Change", "Impression Change", "Max Impression"])
for realSheet, sheetGroup in channelSheets.items():
    existingSheet = next((s for s in sheetGroup if s in available_sheets), None)
    if existingSheet:
        df = pd.read_excel(xls, sheet_name=existingSheet, header=1)
        print(f"\nLoaded sheet {existingSheet} as {realSheet}")
        df = df.dropna(how="all").reset_index(drop=True)
        az = df
        az.columns  = az.columns.to_list()
        az.columns.values[0] = "Page"

        pages = az['Page']
        pages = pages.to_list()

        for row, page in enumerate(pages):
          if isinstance(page, str) and "Assignment" in page:
            cut = row
            break

        if cut is not None:
          channelData = az.iloc[:cut].copy()
          assignmentData = az.iloc[cut:].copy()
          assignmentData.columns = assignmentData.iloc[0]
          assignmentData = assignmentData[1:].reset_index(drop=True)
        else:
            channelData = az
            assignmentData = None

        channelData["Platform"] = None
        channelData["Followers"] = channelData["Followers"].replace("-", None)

        keywords = {
            "Facebook": ["Facebook"],
            "Twitter": ["Twitter (X)"],
            "Instagram": ["Instagram"],
            "TikTok": ["TikTok"],
            "Website": ["Website", "Websites"],
            "Youtube": ["Youtube"],
            "Weibo": ["Weibo"],
            "Azaad Media Academy": ["Azaad Media Academy"]
        }

        keyword_map = {}
        for canonical, variants in keywords.items():
            for v in variants:
                keyword_map[v.lower()] = canonical

        pages_lower = channelData["Page"].str.lower()
        channelData["Platform"] = pages_lower.where(pages_lower.isin(keyword_map))
        channelData["Platform"] = channelData["Platform"].ffill()
        channelData["Platform"] = channelData["Platform"].map(keyword_map)
        channelData = channelData[~pages_lower.isin(keyword_map)].reset_index(drop=True)


        channelData.columns = channelData.columns.str.replace(r"\s+", " ", regex=True).str.strip()
        daysData = sum("Impressions" in col for col in channelData.columns)
        print("Number of headers with 'Impressions':", daysData)

        impressionsHeaders = [col for col in channelData.columns if "Impressions" in col]
        print(impressionsHeaders)
        for index, colImp in enumerate(impressionsHeaders):
            parts = colImp.split("(")
            date = parts[1].strip(")")
            totalPostsColName = f"Total Posts ({date})"
            if index in (0, 1):
                print(f"ignoring the data of {date} at index: {index}")
                continue
            else:

                if totalPostsColName in channelData.columns and colImp in channelData.columns:
                    channelData[[colImp, totalPostsColName]] = channelData[[colImp, totalPostsColName]].replace("-", None).fillna(0)
                    print(f"{realSheet}, Found: {totalPostsColName} and {colImp}")
                    followersCol = channelData["Followers"] if index == daysData - 1 else None
                    imp_cols = channelData[impressionsHeaders[:index+1]].apply(pd.to_numeric, errors="coerce")
                    maxImp = imp_cols.max(axis=1)
                    secondMaxImp = imp_cols.apply(lambda row: row.nlargest(2).iloc[-1] if row.notna().sum() > 1 else None, axis=1)
                    impChange = []
                    for i in channelData.index:
                        current = channelData.loc[i, colImp]
                        max_val = maxImp.loc[i]
                        second_max_val = secondMaxImp.loc[i]

                        if max_val == 0:
                            if current == 0:
                                impChange.append(0)
                            else:
                                impChange.append(1)
                        elif current == max_val:
                            impChange.append((current - second_max_val) / second_max_val)
                        else:
                            impChange.append((current - max_val) / max_val)

                    prevDatePosts = impressionsHeaders[index - 1]
                    prevDateParts =  prevDatePosts.split("(")
                    prevDate = prevDateParts[1].strip(")")
                    prevDatePostData = f"Total Posts ({prevDate})"
                    postChange = []
                    for i in channelData.index:
                        current = channelData.loc[i, totalPostsColName]
                        prev = channelData.loc[i, prevDatePostData]
                        if prev == 0:
                            if current == 0:
                                postChange.append(0)
                            else:
                                postChange.append(1)
                        else:
                            postChange.append((current - prev) / prev)

                    newData = pd.DataFrame({
                        "Page": channelData["Page"],
                        "Total Posts": channelData[totalPostsColName],
                        "Impressions": channelData[colImp],
                        "Followers": followersCol,
                        "Channel": realSheet,
                        "Platform": channelData["Platform"],
                        "Date": date,
                        "Max Impression": maxImp,
                        "Post Change": postChange,
                        "Impression Change": impChange
                    })
                    formattedChannelData = pd.concat([formattedChannelData, newData], ignore_index=True)
                else:
                    print(f"NOT FOUND: {totalPostsColName} or {colImp}")
                    break
    else:
        print(f"Channel not found {realSheet}")
from google.colab import files
from openpyxl.styles import Alignment, Border, Side
from openpyxl.utils import get_column_letter

with pd.ExcelWriter("ChannelDataFormatted.xlsx", engine="openpyxl") as writer:
    formattedChannelData.to_excel(writer, sheet_name="Channel", index=False)

    workbook = writer.book
    thin_border = Border(
        left=Side(style="thin"),
        right=Side(style="thin"),
        top=Side(style="thin"),
        bottom=Side(style="thin")
    )

    for sheetname in writer.sheets:
        worksheet = writer.sheets[sheetname]

        max_row = worksheet.max_row
        max_col = worksheet.max_column

        for row in worksheet.iter_rows(min_row=1, max_row=max_row, min_col=1, max_col=max_col):
            for cell in row:
                cell.alignment = Alignment(horizontal="center", vertical="center")
                cell.border = thin_border

        for col in range(1, max_col + 1):
            col_letter = get_column_letter(col)
            worksheet.column_dimensions[col_letter].width = 25

        for row in range(1, max_row + 1):
            worksheet.row_dimensions[row].height = 25

files.download("ChannelDataFormatted.xlsx")






Loaded sheet Azaad Digital as Azaad Digital
Number of headers with 'Impressions': 4
['Impressions (8-Sep-2025)', 'Impressions (9-Sep-2025)', 'Impressions (10-Sep-2025)', 'Impressions (11-Sep-2025)']
ignoring the data of 8-Sep-2025 at index: 0
ignoring the data of 9-Sep-2025 at index: 1
Azaad Digital, Found: Total Posts (10-Sep-2025) and Impressions (10-Sep-2025)
Azaad Digital, Found: Total Posts (11-Sep-2025) and Impressions (11-Sep-2025)

Loaded sheet Pukhtun Digital as Pukhtun Digital
Number of headers with 'Impressions': 4
['Impressions (8-Sep-2025)', 'Impressions (9-Sep-2025)', 'Impressions (10-Sep-2025)', 'Impressions (11-Sep-2025)']
ignoring the data of 8-Sep-2025 at index: 0
ignoring the data of 9-Sep-2025 at index: 1
Pukhtun Digital, Found: Total Posts (10-Sep-2025) and Impressions (10-Sep-2025)
Pukhtun Digital, Found: Total Posts (11-Sep-2025) and Impressions (11-Sep-2025)

Loaded sheet Kashmir Digital as Kashmir Digital
Number of headers with 'Impressions': 4
['Impressions (

/tmp/ipython-input-3705557979.py:101: RuntimeWarning: divide by zero encountered in scalar divide
  impChange.append((current - second_max_val) / second_max_val)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>